# **Chapter 2. Fundamentals**
---
---

In Chapter 1, I described th major conceptual building block for understanding deep learning: nasted continuous, differentiable functions. I showed how to represent these functions as computational graphs, with each node in a graph representating a single, simple function. In particular, I demonstrated that such a representation showed easily how to calculate the derivative of the output of the nasted function with respect to its input: we simply take the derivatives of all the constituent functions, evaluate these derivatives as the input that these functions recived, and then multiply all of the results together; this will result in a correct derivative for the nested functions because of the chain rule. I illustrated that this does in fact work with some simple examples, with functions that took `NumPy's ndarrays` as inputs and produced `ndarrays` as outputs.

I showed that this method of computing derivatives works even when the function takes in multiple `ndarrays` as inputs and combines them via a *matrix multiplication* operation, which, unlike the other operations we saw, changes the shape of its inputs. Specifically, if one input to this operation—call the input $X$—is a `B × N ndarray`, and another input to this operation, $W$, is an `N × M ndarray`, then its output $P$ is a `B × M ndarray`. While it isn’t clear what the derivative of such an operation would be, I showed that when a matrix multiplication $v(X, W)$ is included as a “constituent operation” in a nested function, we can still use a simple expression *in place of* its derivative to compute the derivatives of its inputs: specifically, the role of $\frac{\partial v}{\partial u}(W)$ can be filled by $X^T$, and the role of $\frac{\partial v}{\partial u}(X)$ can be played by $W^T$.

In this chapter, we’ll start translating these concepts into real-world applications, Specifically, we will:

1. Express linear regression in terms of these building blocks
2. Show that the reasoning around derivatives that we did in Chapter 1 allows us to train this linear regression model
3. Extend this model (still using our building blocks) to a one-layer neural network

Then, in Chapter 3, it will be straightforward to use these same building blocks to build deep learning models.

Before we dive into all this, though, let’s give an overview of *supervised learning*, the subset of machine learning that we’ll focus on as we see how to use neural networks to solve problems.

## # Supervised Learning Overview
---

At a high level, machine learning can be described as building algorithms that can uncover or “learn” *relationships* in data; supervised learning can be described as the subset of machine learning dedicated to finding relationships *between characteristics of the data that have already been measured.*

In this chapter, we'll deal with a typical supervised learning problem that you might encounter in the real world: finding the relationship between characteristics of a house and the value of the house. Clearly, there is some relationship between characteristics such as the number of rooms, the square footage, or the proximity to schools and how desirable a house is to live in or own. At a high level, the aim of supervised learning is to uncover these relationships, given that we've *already measured* these characteristics.

By "measure", I mean that each characteristic has been defined precisely and represented as a number. Many characteristics of a house, such as the number of bedrooms, the square footage, and so on, naturally lend themselves to being represented as numbers, but if we had other, different kinds of information, such as natural language descriptions of the house’s neighborhood from TripAdvisor, this part of the problem would be much less straightforward, and doing the translation of this less-structured data into numbers in a reasonable way could make or break our ability to uncover relationships. In addition, for any concept that is ambiguously defined, such as the value of a house, we simply have to pick a single number to describe it; here, an obvious choice is to use the price of the house.

Once we’ve translated our “characteristics” into numbers, we have to decide what structure to use to represent these numbers. One that is nearly universal across machine learning and turns out to make computations easy is to represent each set of numbers for a single observation—for example, a single house—as a row of data, and then stack these rows on top of each other to form “batches” of data that will get fed into our models as two-dimensional `ndarrays`. Our models will then return predictions as output `ndarrays` with each prediction in a row, similarly stacked on top of each other, with one prediction for each observation in the batch.

Now for some definitions: we say that the length of each row in this `ndarray` is the number of *features* of our data. In general, a single characteristic can map to many features, a classic example being a characteristic that describes our data as belonging to one of several *categories*, such as being a red brick house, a tan brick house, or a slate house; in this specific case we might describe this single characteristic with three features. The process of mapping what we informally think of as characteristics of our observations into features is called *feature engineering*. I won’t spend much time discussing this process in this book; indeed, in this chapter we’ll deal with a problem in which we have 13 characteristics of each observation, and we simply represent each characteristic with a single numeric feature.

I said that the goal of supervised learning is ultimately to uncover relationships between characteristics of data. In practice, we do this by choosing one characteristic that we want to predict from the others; we call this characteristic our *target*. The choice of which characteristic to use as the target is completely arbitrary and depends on the problem you are trying to solve. For example, if your goal is just to *describe* the relationship between the prices of houses and the number of rooms they have, you could do this by training a model with the prices of houses as the target and the number of rooms as a feature, or vice versa; either way, the resulting model will indeed contain a description of the relationship between these two characteristics, allowing you to say, for example, a higher number of rooms in a house is associated with higher prices. On the other hand, if your goal is to *predict* the prices of houses *for which no price information is available*, you have to choose the price as your target, so that you can ultimately feed the other information into your model once it is trained.

Figure 2-1 shows this hierarchy of descriptions of supervised learning, from the highest-level description of finding relationships in data, to the lowest level of quantifying those relationships by training models to uncover numerical representations between the features and the target.

<div align="center">
    <img src="https://learning.oreilly.com/library/view/deep-learning-from/9781492041405/assets/dlfs_0201.png" width="600px"/>
    <i>Figure 2-1. Supervised learning overview</i>
</div>

As mentioned, we’ll spend almost all our time on the level highlighted at the bottom of Figure 2-1; nevertheless, in many problems, getting the parts at the top correct—collecting the right data, defining the problem you are trying to solve, and doing feature engineering—is much harder than the actual modeling. Still, since this book is focused on modeling—specifically, on understanding how deep learning models work—let’s return to that subject.